# Coulsen tables
Runs the Coulsen tables

In [1]:
import sasktran2 as sk
import numpy as np
import xarray as xr
import seaborn as sns 

colors = sns.color_palette('colorblind')

mus = np.array([0.02, 0.06, 0.1, 0.16, 0.2, 0.28, 0.32, 0.4, 0.52, 0.64, 0.72, 0.84, 0.92, 0.96, 0.98, 1.0])
phis = np.array([1e-4, 30, 60, 90, 120, 150, 180])

def run_for_condition(num_alt, cos_sza, albedo, tau):
    top_alt = 100001
    config = sk.Config()
    config.multiple_scatter_source = sk.MultipleScatterSource.DiscreteOrdinates
    config.single_scatter_source = sk.SingleScatterSource.DiscreteOrdinates
    config.init_successive_orders_with_discrete_ordinates = False
    config.num_successive_orders_iterations = 50
    config.num_successive_orders_outgoing = 110
    config.num_successive_orders_incoming = 110

    # config.single_scatter_source = sk.SingleScatterSource.NoSource
    config.num_stokes = 3
    config.num_streams = 40
    config.num_singlescatter_moments = 40

    model_geometry = sk.Geometry1D(cos_sza=cos_sza,
                                    solar_azimuth=0,
                                    earth_radius_m=63720000,
                                    altitude_grid_m=np.linspace(0, top_alt, num_alt, endpoint=True),
                                    interpolation_method=sk.InterpolationMethod.LinearInterpolation,
                                    geometry_type=sk.GeometryType.PlaneParallel)

    viewing_geo = sk.ViewingGeometry()

    for mu in mus:
        for phi in phis:
            ray = sk.GroundViewingSolar(cos_sza, np.pi - (phi) * np.pi/180, mu, top_alt*2)
            viewing_geo.add_ray(ray)

    atmosphere = sk.Atmosphere(model_geometry, config, calculate_derivatives=False, numwavel=1)


    atmosphere.storage.total_extinction[:] = tau / top_alt
    atmosphere.storage.ssa[:] = 1

    atmosphere.leg_coeff.a1[0, :, 0] = 1
    atmosphere.leg_coeff.a1[2, :, 0] = 0.5

    atmosphere.leg_coeff.a2[2] = 3
    atmosphere.leg_coeff.b1[2] = np.sqrt(6.0) / 2

    atmosphere.surface.albedo[:] = albedo

    engine = sk.Engine(config, model_geometry, viewing_geo)

    return engine.calculate_radiance(atmosphere)

test = run_for_condition(2, 0.2, 0, 0.5)
test['radiance'] * np.pi

<xarray.DataArray 'radiance' (wavelength: 1, los: 112, stokes: 3)>
array([[[ 4.41293240e-01,  1.75261058e-02,  1.55014156e-07],
        [ 3.94444790e-01,  6.48478175e-02,  4.39036280e-02],
        [ 3.00907330e-01,  1.59650679e-01,  7.36552548e-02],
        [ 2.54653913e-01,  2.07567424e-01,  8.12829559e-02],
        [ 3.02532989e-01,  1.61276339e-01,  6.71309546e-02],
        [ 3.97260515e-01,  6.76635417e-02,  3.73793279e-02],
        [ 4.44544558e-01,  2.07774240e-02,  9.03168968e-18],
        [ 3.92505813e-01,  1.77156658e-02,  1.59193697e-07],
        [ 3.52229443e-01,  5.94372415e-02,  4.42873579e-02],
        [ 2.72097636e-01,  1.43301327e-01,  7.04698002e-02],
        [ 2.33392212e-01,  1.86593849e-01,  7.15317627e-02],
        [ 2.76389542e-01,  1.47593233e-01,  5.34268472e-02],
        [ 3.59663242e-01,  6.68710404e-02,  2.72444048e-02],
        [ 4.01089625e-01,  2.62994773e-02,  6.35006989e-18],
        [ 3.50826761e-01,  1.52399156e-02,  1.61626201e-07],
        [ 3.15752154e-01,  5.27437564e-02,  4.43577622e-02],
        [ 2.46226447e-01,  1.28374944e-01,  6.76273195e-02],
        [ 2.13478797e-01,  1.68205740e-01,  6.35736128e-02],
        [ 2.52583809e-01,  1.34732305e-01,  4.24854079e-02],
        [ 3.26763427e-01,  6.37550291e-02,  1.92158506e-02],
...
        [ 5.37256532e-02, -2.05347669e-02,  3.47592962e-02],
        [ 5.39196075e-02,  1.85283477e-02,  3.65484893e-02],
        [ 5.55040919e-02,  3.95474121e-02,  4.88816643e-03],
        [ 5.86122473e-02,  2.32209874e-02, -2.80819367e-02],
        [ 6.18535437e-02, -1.24068764e-02, -2.98711298e-02],
        [ 6.32440738e-02, -3.04509265e-02, -8.54076286e-18],
        [ 5.24041035e-02, -3.96992703e-02,  1.36645347e-07],
        [ 5.24742308e-02, -2.05277882e-02,  3.41301796e-02],
        [ 5.29426889e-02,  1.81433794e-02,  3.53800487e-02],
        [ 5.42376879e-02,  3.85397333e-02,  3.41470585e-03],
        [ 5.62891006e-02,  2.14897912e-02, -2.94656046e-02],
        [ 5.82703859e-02, -1.47316331e-02, -3.07154737e-02],
        [ 5.90969269e-02, -3.30064469e-02, -8.75164504e-18],
        [ 5.30059306e-02, -3.75585903e-02,  4.45747681e-07],
        [ 5.30059309e-02,  1.06822571e-02,  3.60074586e-02],
        [ 5.30059309e-02, -3.22381010e-03,  3.74199778e-02],
        [ 5.30059309e-02,  3.75585901e-02, -1.20797639e-17],
        [ 5.30059309e-02,  2.05879432e-02, -3.14131228e-02],
        [ 5.30059309e-02, -2.37561380e-03, -3.74833850e-02],
        [ 5.30059309e-02, -3.75585901e-02, -9.19920143e-18]]])
Coordinates:
  * stokes   (stokes) <U1 'I' 'Q' 'U'
Dimensions without coordinates: wavelength, los